<a href="https://colab.research.google.com/github/andyharless/twit_demog/blob/master/twitgen_use_large_make_embed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
do_mount=True
if do_mount:
  from google.colab import drive
  drive.mount('/content/gdrive')
  !pip install "tensorflow-gpu==1.13.1"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
     |████████████████████████████████| 345.2MB 69kB/s 
     |████████████████████████████████| 368kB 51.1MB/s 
     |████████████████████████████████| 3.2MB 49.1MB/s 
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existin

In [0]:
do_save_embeddings = True

CORPUS = 'twitgen_big_corpus_201907251843.csv'
EMBEDDING_DIM = 512

In [0]:
# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
import tensorflow.keras.layers as layers
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Layer
from tensorflow.keras.optimizers import Adam, Adagrad
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc
import numpy as np
from datetime import datetime

In [0]:
basepath = '/content/gdrive/My Drive/twitgen/'
model_file_name = 'USELModel_201906161241.h5'

In [0]:
df = pd.read_csv(basepath+CORPUS, index_col=['id','time'], parse_dates=['time'])
df.head()

,,text,male
id,time,,
1703564846,2019-05-21 17:50:48+00:00,I prefer tubs.. but nice try jiggly,True
211806644,2019-05-21 17:50:48+00:00,It's the shop. I'd never have paid for 2 year...,False
971515498411241472,2019-05-21 17:50:48+00:00,Why can’t attend Cannes 🎬,False
2430359987,2019-05-21 17:50:49+00:00,"Raspberry gin and tonic, nomnom 😋 beautiful ev...",False
2860188236,2019-05-21 17:50:49+00:00,This is 🏴󠁧󠁢󠁥󠁮󠁧󠁿🇬🇧🏴󠁧󠁢󠁥󠁮󠁧󠁿🇬🇧🏴󠁧󠁢󠁥󠁮󠁧󠁿 here we g...,True


In [0]:
# Create dataset
texts = df['text'].tolist()
texts = [' '.join(t.split()[0:150]) for t in texts]
texts = np.array(texts, dtype=object)[:, np.newaxis]
labels = df['male'].tolist()

In [0]:
# Initialize session
tf.logging.set_verbosity(tf.logging.ERROR)
sess = tf.Session()
K.set_session(sess)

In [0]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class USEEmbeddingLayer(Layer):
    def __init__(self, trainable=True, **kwargs):
        self.dimensions = EMBEDDING_DIM
        super(USEEmbeddingLayer, self).__init__(**kwargs)
        self.trainable=trainable

    def build(self, input_shape):
        self.use = hub.Module('https://tfhub.dev/google/universal-sentence-encoder-large/3', trainable=True, #self.trainable,
                               name="{}_module".format(self.name))
        wts = tf.trainable_variables(scope=".*{}_module/.*".format(self.name))
        wts = [v for v in wts if not ("SNLI" in v.name or "SHARED_RANK_ANSWER" in v.name)]
        if len(wts)>0:
          if self.trainable:
            self._trainable_weights += wts
          else:
            self._non_trainable_weights += wts
        else:
          print('WARNING: No weights in Embedding Layer')
        super(USEEmbeddingLayer, self).build(input_shape)

   
    def call(self, x, mask=None):
        result = self.use(tf.squeeze(tf.cast(x, tf.string), axis=1))
        return result

    def compute_mask(self, inputs, mask=None):
        return tf.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)


In [0]:
# Function to build model
def build_model(train_embedding=True, optimizer=Adagrad, lr=.003): 
  input_text = layers.Input(shape=(1,), dtype="string")
  embedding = USEEmbeddingLayer(trainable=train_embedding)(input_text)
  dense1 = layers.Dense(512, activation='relu')(embedding)

  dropout0 = layers.Dropout(.8)(dense1)
  pred = layers.Dense(1, activation='sigmoid')(dropout0)

  model = Model(inputs=[input_text], outputs=pred)

  model.compile(loss='binary_crossentropy', optimizer=optimizer(lr), metrics=['accuracy'])
  model.summary()
  
  return model

In [0]:
sess = tf.Session()
K.set_session(sess)
model = build_model(train_embedding=True)

init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run([init, table_init])
K.set_session(sess)

lr = 7e-5
model.compile(loss='binary_crossentropy', optimizer=Adam(lr), metrics=['accuracy'])
model.load_weights(basepath+model_file_name)

inp = model.input
embeddings = model.layers[1].output
get_embeddings = K.function([inp],[embeddings])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
use_embedding_layer (USEEmbe (None, 512)               211345728 
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 211,608,897
Trainable params: 211,608,897
Non-trainable params: 0
_________________________________________________________________


In [0]:
chunk_size = 2**13
timestamp = datetime.now().strftime('%Y%m%d%H%M')

In [0]:
def save_embeddings(df, split, timestamp):

  full_len = df.shape[0]
  tweet_embeddings = np.empty(shape=[0,EMBEDDING_DIM])

  for start in range(0, full_len, chunk_size):
    end = min(start+chunk_size, full_len)
    embeddings_chunk = get_embeddings([df[['text']].values[start:end]])[0]
    tweet_embeddings = np.concatenate([tweet_embeddings, embeddings_chunk])

  for i in range(tweet_embeddings.shape[1]):
    df['embed'+str(i)] = (tweet_embeddings[:,i]*1e6).astype(int)
  
  embed_file_name = basepath + 'embed_' + split + '_' + timestamp + '.csv'
  df.drop(['text'],axis=1).to_csv(embed_file_name)

In [0]:
save_embeddings(df, 'corpus', timestamp)

In [0]:
timestamp

'201907252332'

In [0]:
embed_file_name = basepath + 'embed_' + 'corpus' + '_' + timestamp + '.csv'
!ls '$embed_file_name'

'/content/gdrive/My Drive/twitgen/embed_corpus_201907252332.csv'
